# -----------------PROYECTO FINAL-----------------

## 1. IMPORTAR LIBRERIAS

In [44]:
from keras.src.saving.saving_api import load_model
import cv2
import mediapipe as mp
import numpy as np
import os
import tensorflow as tf
import sys
from collections import deque  

## 2. INICIALIZAR MEDIAPIPE

In [45]:
# Configuración inicial global
mp_hands = mp.solutions.hands

# Optimizar MediaPipe
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.45,  # Reducir confianza
    min_tracking_confidence=0.45,
    model_complexity=0  # Menor complejidad
)

mp_draw = mp.solutions.drawing_utils
dataset_dir = "dataset_11_90"
data_dir_video  = "dataset_11_90_video"
model_path = "gesture_model_me_10_90.h5"

sequence_length = 90
total_landmarks = 126
gestures = []
X_mean = None
X_std = None

num_camara = 0

## 3. FUNCIONES PRINCIPALES

In [46]:
# Funciones principales
def init_system():
    global gestures
    os.makedirs(dataset_dir, exist_ok=True)
    gestures = get_existing_gestures()
    
def get_existing_gestures():
    valid_gestures = []
    for d in os.listdir(dataset_dir):
        full_path = os.path.join(dataset_dir, d)
        if os.path.isdir(full_path):
            if any(f.endswith('.npy') for f in os.listdir(full_path)):
                valid_gestures.append(d)
            else:
                print(f"¡Directorio vacío: {d}!")
    return valid_gestures

## Funciones auxiliares
def process_frame(frame):
    """Procesa el frame para detectar manos con MediaPipe."""
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    return results

def extract_landmarks(hands_results, max_hands=2, landmarks_per_hand=21):
    """Extrae los landmarks de las manos detectadas y rellena con ceros si no se detectan manos."""
    landmarks = []
    if hands_results.multi_hand_landmarks:
        for hand_landmarks in hands_results.multi_hand_landmarks[:max_hands]:  # Limitar al número máximo de manos
            landmarks.extend([(lm.x, lm.y, lm.z) for lm in hand_landmarks.landmark])
    
    # Rellenar con ceros si no se detectan suficientes manos
    expected_length = max_hands * landmarks_per_hand * 3  # 3 valores (x, y, z) por landmark
    if len(landmarks) < expected_length:
        landmarks.extend([(0.0, 0.0, 0.0)] * (expected_length - len(landmarks)))
    
    return landmarks

## 4. DETECCION DE MANO

def detect_hands():
    print("\nIniciando detección de manos. Presiona 'ESC' para salir.")
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        #results = hands.process(rgb_frame)

        frame = cv2.flip(frame, 1)
        results = process_frame(frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Detección de Manos", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [47]:
# Funciones auxiliares
def process_frame(frame):
    """Procesa el frame para detectar manos con MediaPipe."""
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)
    return results

def extract_landmarks(hands_results, max_hands=2, landmarks_per_hand=21):
    """Extrae los landmarks de las manos detectadas y rellena con ceros si no se detectan manos."""
    landmarks = []
    if hands_results.multi_hand_landmarks:
        for hand_landmarks in hands_results.multi_hand_landmarks[:max_hands]:  # Limitar al número máximo de manos
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])  # Añadir cada coordenada como elemento individual
    
    # Rellenar con ceros si no se detectan suficientes manos
    expected_length = max_hands * landmarks_per_hand * 3  # 3 valores (x, y, z) por landmark
    if len(landmarks) < expected_length:
        landmarks.extend([0.0] * (expected_length - len(landmarks)))  # Rellenar con 0.0s
    
    return landmarks

# Función para detectar manos en tiempo real
def detect_hands():
    print("\nIniciando detección de manos. Presiona 'ESC' para salir.")
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        results = process_frame(frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        cv2.imshow("Detección de Manos", frame)
        if cv2.waitKey(1) & 0xFF == 27:  # Presiona ESC para salir
            break

    cap.release()
    cv2.destroyAllWindows()

## 5. RECOLLECION DE DATOS 

In [48]:
# Función para recolectar datos de gestos
def collect_data(data_dir_video, dataset_dir, sign_name, sequence_length):
    """Recolecta secuencias de movimiento para una seña específica y guarda el video de los landmarks."""
    sign_dir = os.path.join(dataset_dir, sign_name)
    sign_dir_video = os.path.join(data_dir_video, f"{sign_name}")
    os.makedirs(sign_dir, exist_ok=True)
    os.makedirs(sign_dir_video, exist_ok=True)

    cap = cv2.VideoCapture(0)  # Cambiar índice si usas DroidCam u otra cámara
    if not cap.isOpened():
        print("Error: No se pudo abrir la cámara")
        return

    total_sequences = int(input("Número de secuencias a recolectar (recomendado: 20-30): "))

    print("\nInstrucciones:")
    print(f"1. Cada secuencia grabará {sequence_length} frames de movimiento")
    print("2. Presiona ESPACIO para iniciar cada secuencia")
    print("3. Realiza el movimiento completo de la seña")
    print("4. La grabación se detendrá automáticamente")
    print("5. Presiona ESC para cancelar")

    sequence_count = 0
    frame_count = 0
    is_recording = False
    current_sequence = []

    # Configuración para guardar el video
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = 30  # Puedes ajustar los FPS según sea necesario
    video_writer = None

    while sequence_count < total_sequences:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        blank_frame = np.zeros_like(frame)
        hands_results = process_frame(frame)

        # Dibujar landmarks de las manos
        if hands_results.multi_hand_landmarks:
            for hand_landmarks in hands_results.multi_hand_landmarks:
                mp_draw.draw_landmarks(blank_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        # Mostrar mensajes
        if is_recording:
            cv2.putText(blank_frame, f"Grabando secuencia {sequence_count + 1}...", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            landmarks = extract_landmarks(hands_results)
            current_sequence.append(landmarks)
            frame_count += 1

            if frame_count >= sequence_length:
                # Guardar la secuencia de landmarks
                sequence_data = np.array(current_sequence)
                np.save(os.path.join(sign_dir, f"sequence_{sequence_count}.npy"), sequence_data)

                # Guardar el video
                video_writer.release()
                print(f"Secuencia {sequence_count + 1}/{total_sequences} guardada")
                sequence_count += 1
                frame_count = 0
                is_recording = False
                current_sequence = []

        cv2.imshow("Recolección de Datos", frame)  # Mostrar el video original
        cv2.imshow("Landmarks", blank_frame)  # Mostrar los landmarks

        key = cv2.waitKey(1) & 0xFF
        if key == 32 and not is_recording:  # Espacio para iniciar grabación
            is_recording = True
            current_sequence = []
            frame_count = 0
            # Inicializar el VideoWriter para guardar el video
            video_path = os.path.join(sign_dir_video, f"sequence_{sequence_count}.avi")
            video_writer = cv2.VideoWriter(video_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (frame_width, frame_height))
        elif key == 27:  # ESC para salir
            break

        # Escribir el frame en el video si está grabando
        if is_recording:
            video_writer.write(blank_frame)  # Guardar solo los landmarks

    cap.release()
    if video_writer is not None:
        video_writer.release()
    cv2.destroyAllWindows()

## 6. CARGA DE DATOS

In [49]:
def load_data():
    X, y = [], []
    for label_idx, gesture in enumerate(gestures):
        gesture_dir = os.path.join(dataset_dir, gesture)
        if not os.path.exists(gesture_dir):
            print(f"¡Error! Directorio no encontrado: {gesture}")
            continue
            
        sequences = [f for f in os.listdir(gesture_dir) if f.endswith('.npy')]
        if not sequences:
            print(f"¡No hay archivos .npy en {gesture}!")
            continue
            
        
        for seq_file in sequences:
            seq_path = os.path.join(gesture_dir, seq_file)
            try:
                sequence = np.load(seq_path)
                if sequence.shape == (sequence_length, total_landmarks):
                    X.append(sequence)
                    y.append(label_idx)
                else:
                    print(f"¡Secuencia inválida en {gesture}: {seq_file}!")
            except Exception as e:
                print(f"Error cargando {seq_path}: {str(e)}")
    
    if len(X) == 0 or len(y) == 0:
        print("\nERROR CRÍTICO: Dataset vacío o corrupto.")
        print("Posibles causas:")
        print("- Directorios de gestos vacíos")
        print("- Archivos .npy con formato incorrecto")
        print("- Secuencias con dimensiones diferentes a (sequence_length, total_landmarks)")
        return np.array([]), np.array([]), gestures
    
    return np.array(X), np.array(y), gestures


## 7. ENTRENAMIENTO DEL MODELO

In [50]:
def train_model():
    global X_mean, X_std, gestures
    
    # 1. Verificar datos de entrenamiento
    #gestures = get_existing_gestures()
    if not gestures:
        print("\nNo hay datos recolectados. Primero recolecte datos de gestos.")
        return

    # 2. Cargar y preparar datos
    print("\nCargando datos y preparando el entrenamiento...")
    X, y, gestures = load_data()
    y = tf.keras.utils.to_categorical(y)

    # 3. Calcular parámetros de normalización
    X_mean = np.mean(X, axis=(0, 1))
    X_std = np.std(X, axis=(0, 1))
    X = (X - X_mean) / X_std  # Normalización

    # 4. Guardar parámetros de normalización
    np.savez('normalization_params_90.npz', mean=X_mean, std=X_std)
    
    # 5. Arquitectura optimizada del modelo
    inputs = tf.keras.Input(shape=(sequence_length, total_landmarks))
    x = tf.keras.layers.Conv1D(64, 3, activation='relu', padding='same')(inputs)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x)
    outputs = tf.keras.layers.Dense(len(gestures), activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # 6. Compilación y entrenamiento
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nIniciando entrenamiento...")
    history = model.fit(
        X, y,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)],
        verbose=1
    )

    # 7. Guardar modelo y resultados
    model.save(model_path)
    print(f"\nModelo guardado en {model_path}")
    
    # 8. Conversión a TFLite con configuraciones especiales
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [
        tf.lite.OpsSet.TFLITE_BUILTINS,
        tf.lite.OpsSet.SELECT_TF_OPS
    ]
    converter._experimental_lower_tensor_list_ops = False
    
    try:
        tflite_model = converter.convert()
        with open('model_quantized_90.tflite', 'wb') as f:
            f.write(tflite_model)
        print("\nModelo TFLite exportado exitosamente")
    except Exception as e:
        print(f"\nError en conversión TFLite: {str(e)}")
    
    # Mostrar métricas finales
    val_accuracy = history.history['val_accuracy'][-1]
    print(f"Precisión de validación final: {val_accuracy:.2%}")

## TF LITE

In [51]:
def convert_to_tflite():
    try:
        # Cargar el modelo entrenado
        model = tf.keras.models.load_model(model_path)
        
        # Configurar el conversor con parámetros especiales
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        
        # Añadir estas 3 líneas clave para compatibilidad con LSTM
        converter.target_spec.supported_ops = [
            tf.lite.OpsSet.TFLITE_BUILTINS,
            tf.lite.OpsSet.SELECT_TF_OPS
        ]
        converter._experimental_lower_tensor_list_ops = False
        converter.allow_custom_ops = True  # Permitir operaciones personalizadas
        
        # Realizar la conversión
        tflite_model = converter.convert()
        
        # Guardar el modelo cuantizado
        with open('model_quantized.tflite', 'wb') as f:
            f.write(tflite_model)
            
        print("\n✅ Conversión a TFLite exitosa!")
        
    except Exception as e:
        print(f"\n❌ Error en conversión: {str(e)}")
        print("Posibles soluciones:")
        print("1. Verifique que el modelo .h5 existe")
        print("2. Actualice TensorFlow: pip install --upgrade tensorflow")
        print("3. Reinicie el runtime/kernel")

def representative_dataset_gen():
    # Generador de datos de ejemplo para calibración
    for _ in range(100):
        yield [np.random.randn(1, sequence_length, total_landmarks).astype(np.float32)]

## 8. EVALUACION DEL MODELO

In [52]:
# ------------------ IMPORTS ADICIONALES NECESARIOS ------------------
from collections import deque
from threading import Thread
from queue import Queue


# ------------------ SECCIÓN DE CONFIGURACIÓN ------------------
# Añadir al inicio con las demás configuraciones
frame_queue = Queue(maxsize=30)
landmark_queue = Queue(maxsize=20)
prediction_queue = Queue(maxsize=10)

# ------------------ FUNCIÓN CAMERA_CAPTURE ------------------
def camera_capture(cap):
    while True:
        ret, frame = cap.read()
        if ret:
            resized_frame = cv2.resize(frame, (320, 240))
            frame_queue.put(resized_frame)

# ------------------ FUNCIÓN LANDMARK_PROCESSING ------------------
def landmark_processing():
    while True:
        frame = frame_queue.get()
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        all_landmarks = np.zeros(total_landmarks, dtype=np.float32)
        if results.multi_hand_landmarks:
            idx = 0
            for hand in results.multi_hand_landmarks[:2]:
                for lm in hand.landmark:
                    all_landmarks[idx:idx+3] = [lm.x, lm.y, lm.z]
                    idx += 3
        landmark_queue.put(all_landmarks)

# ------------------ FUNCIÓN MODEL_INFERENCE ------------------
def model_inference(interpreter, input_details, output_details):
    while True:
        data = landmark_queue.get()
        if data is not None:
            sequence.append(data)
            sequence = sequence[-sequence_length:]
            
            if len(sequence) == sequence_length:
                seq_array = (np.array(sequence) - X_mean) / X_std
                input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                
                interpreter.set_tensor(input_details[0]['index'], input_data)
                interpreter.invoke()
                prediction = interpreter.get_tensor(output_details[0]['index'])[0]
                
                predicted_class = np.argmax(prediction)
                confidence = np.max(prediction)
                prediction_queue.put((gestures[predicted_class], confidence))



In [53]:
from threading import Thread
def evaluate():
    if not os.path.exists("model_quantized_90.tflite"):
        print("\n¡Primero debe entrenar y convertir el modelo!")
        return
    
    # 1. Cargar parámetros y modelo
    try:
        with np.load('normalization_params_90.npz') as data:
            X_mean = data['mean']
            X_std = data['std']
            
        interpreter = tf.lite.Interpreter(model_path="model_quantized_90.tflite")
        interpreter.allocate_tensors()
        input_details = interpreter.get_input_details()[0]  # Mantener [0]
        output_details = interpreter.get_output_details()[0]
    except Exception as e:
        print(f"\nError crítico: {str(e)}")
        return

    # 2. Configuración de cámara
    cap = cv2.VideoCapture(0)  # Forzar backend DirectShow
    #import time
    #time.sleep(2)  # Permitir inicialización de cámara
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    #cap.set(cv2.CAP_PROP_FPS, 30)
    
    if not cap.isOpened():
        print("\n¡No se puede acceder a la cámara!")
        return

    # 3. Variables de estado
    current_gesture = None  # Variable para almacenar la última seña detectada
    current_confidence = 0.0
    sequence = deque(maxlen=sequence_length)  # Usar deque
    #last_gesture = None
    frame_counter = 0
    #prediction_active = False

    # 4. Bucle principal
    while True:
        # 4.1 Capturar frame
        ret, frame = cap.read()
        if not ret:
            print("\nError en captura de frame")
            break
            
        frame_counter += 1
        
        # 4.2 Procesamiento cada 2 frames
        if frame_counter % 2 != 0:
            continue
            
        # 4.3 Detección de landmarks
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        if results.multi_hand_landmarks:
            # 4.4 Extraer landmarks
            landmarks = []
            for hand in results.multi_hand_landmarks[:2]:
                for lm in hand.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])
            
            # Rellenar si solo hay una mano
            if len(results.multi_hand_landmarks) < 2:
                landmarks += [0.0] * (total_landmarks - len(landmarks))
                
            # El deque ya maneja automáticamente el tamaño máximo gracias a maxlen
            sequence.append(landmarks)  # Esto mantendrá solo los últimos 'sequence_length' elementos
            
            
            # 4.5 Dibujar landmarks
            for hand_landmarks in results.multi_hand_landmarks:
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                

                # Añadir verificación de dimensión de entrada
            if input_details['shape'][1] != sequence_length:
                print("\nError: Dimensiones del modelo no coinciden con la configuración")
                return
            
            # 4.6 Realizar predicción cada 1 segundo (90 frames)
            if len(sequence) == sequence_length and frame_counter % 90 == 0:
                try:
                    # Preprocesamiento CORREGIDO
                    seq_array = np.array(sequence)
                    seq_array = (seq_array - X_mean) / (X_std + 1e-7)  # +epsilon para seguridad
                    
                    # Asegurar 3 dimensiones: [batch_size=1, sequence_length, features]
                    input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                    
                    # Debug: Verificar forma
                    
                    print("Forma de entrada:", input_data.shape)  # Debe mostrar (1, 30, 126)

                    # Inferencia
                    interpreter.set_tensor(input_details['index'], input_data)
                    interpreter.invoke()
                    prediction = interpreter.get_tensor(output_details['index'])[0]
                    
                    # Procesar resultados
                    predicted_idx = np.argmax(prediction)
                    confidence = prediction[predicted_idx]
                    
                    if confidence > 0.35:
                        current_gesture = gestures[predicted_idx]  # Actualizar solo si hay alta confianza
                        current_confidence = confidence
                        print("Nueva seña detectada:", current_gesture)
                        
                        # Actualizar UI
                        #cv2.putText(frame, f"{current_gesture} ({confidence:.2%})", 
                        #          (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                        
                        cv2.putText(frame, f"{current_gesture} ({confidence:.2%})", 
                            (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)
                        
                        print("La seña es: ", current_gesture)

                        """# Reproducir voz
                        if current_gesture != last_gesture:
                            threading.Thread(target=voice_system.speak, 
                                           args=(current_gesture,)).start()
                            last_gesture = current_gesture"""
                            
                except Exception as e:
                    print(f"Error en predicción: {str(e)}")
                    current_gesture = None  # Resetear si hay error
                    
        if current_gesture:
            cv2.putText(frame, 
              f"{current_gesture} ({current_confidence:.2%})", 
              (10, 50), 
              cv2.FONT_HERSHEY_SIMPLEX, 
              1, 
              (0, 255, 0), 
              2)

        """# 4.7 Mostrar frame (CORREGIDO)
        # Redimensionar y centrar la ventana
        display_frame = cv2.resize(frame, (640, 480))  # Ajustar a resolución HD
        
        # Obtener dimensiones de la pantalla
        screen_width = 1920  # Ajustar según la resolución del monitor del usuario
        screen_height = 1080
        
        # Calcular posición centrada
        x_pos = (screen_width - 1500) // 2
        y_pos = (screen_height - 1000) // 2
        
        # Crear ventana y posicionar
        cv2.namedWindow("Reconocimiento de Señas", cv2.WINDOW_NORMAL)
        cv2.resizeWindow("Reconocimiento de Señas", 640, 480)
        cv2.moveWindow("Reconocimiento de Señas", x_pos, y_pos)
        
        cv2.imshow("Reconocimiento de Señas", display_frame)"""

        # Mostrar frame reducido
        cv2.imshow("Predicciones", cv2.resize(frame, (640, 480)))
        
        if cv2.waitKey(1) & 0xFF == 27: break
        
        # 4.8 Salir con ESC
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # 5. Limpieza
    cap.release()
    cv2.destroyAllWindows()
    print("\nSistema de evaluación detenido")

## 9. REENTRENAR GESTO

In [54]:
def retrain_gesture():
    global gestures
    if not gestures:
        print("\nNo hay gestos para reentrenar. Primero recolecte datos.")
        return

    print("\nGestos disponibles para reentrenar:")
    for i, gesture in enumerate(gestures):
        print(f"{i+1}. {gesture}")

    try:
        choice = int(input("\nSeleccione el número del gesto a reentrenar: ")) - 1
        if 0 <= choice < len(gestures):
            gesture = gestures[choice]
            gesture_dir = os.path.join(dataset_dir, gesture)
            
            for file in os.listdir(gesture_dir):
                os.remove(os.path.join(gesture_dir, file))
            
            print(f"\nDatos anteriores de '{gesture}' eliminados.")
            collect_data()
            train_model()
        else:
            print("\nSelección inválida.")
    except ValueError:
        print("\nPor favor, ingrese un número válido.")

## 10. MENU

In [55]:
# Menú principal
def main():
    init_system()
    
    while True:
        print("\n=== Sistema de Reconocimiento de Lenguaje de Señas ===")
        print("1. Detectar Manos")
        print("2. Recolectar Datos")
        print("3. Entrenar Modelo, y despues ir a convertir a TFlite")
        print("4. Evaluar")
        print("5. Reentrenar Gesto")
        print("6. Convertir a TFLite")  # Nueva opción
        print("7. Salir")
        
        choice = input("\nSeleccione una opción: ")
        
        if choice == '1':
            detect_hands()
        elif choice == '2':
            sign_name = input("Nombre de la seña a recolectar: ")
            collect_data(data_dir_video, dataset_dir, sign_name, sequence_length)
        elif choice == '3':
            train_model()
        elif choice == '4':
            evaluate()
        elif choice == '5':
            retrain_gesture()
        elif choice == '6':  # Nueva opción de conversión
            convert_to_tflite()
        elif choice == '7':
            print("\n¡Hasta luego!")
            break
        else:
            print("\nOpción inválida. Por favor, intente de nuevo.")

# MENU

In [56]:
if __name__ == "__main__":
    main()


=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Reentrenar Gesto
6. Convertir a TFLite
7. Salir

Cargando datos y preparando el entrenamiento...
¡Secuencia inválida en Hola: sequence_0.npy!
¡Secuencia inválida en Hola: sequence_1.npy!
¡Secuencia inválida en Hola: sequence_10.npy!
¡Secuencia inválida en Hola: sequence_11.npy!
¡Secuencia inválida en Hola: sequence_12.npy!
¡Secuencia inválida en Hola: sequence_13.npy!
¡Secuencia inválida en Hola: sequence_14.npy!
¡Secuencia inválida en Hola: sequence_15.npy!
¡Secuencia inválida en Hola: sequence_16.npy!
¡Secuencia inválida en Hola: sequence_17.npy!
¡Secuencia inválida en Hola: sequence_18.npy!
¡Secuencia inválida en Hola: sequence_19.npy!
¡Secuencia inválida en Hola: sequence_2.npy!
¡Secuencia inválida en Hola: sequence_3.npy!
¡Secuencia inválida en Hola: sequence_4.npy!
¡Secuencia inválida en Hola: sequence_5.npy!
¡Secuencia 

ValueError: zero-size array to reduction operation maximum which has no identity